The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [3]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [83]:
#Basic parameters
## Basic parameters
maxiter=1000;
max_time = 100;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [64]:
dataset = "gisette_scale"   # mnist.scale  a9a  gisette_scale
prob = defineProb(dataset);
showDetails(dataset)

../data/gisette_scale.jld


Dict{Symbol,Any} with 3 entries:
  :sparsity => 0.991
  :dims     => 5000
  :n        => 6000

In [84]:
norm(prob.A*prob.xsol - prob.b)

3.554375254672887e-10

In [85]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [86]:
# Testing Rademarcher Sketch
method_name = "rademacher";# ROS
options.sketchsize =10;
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-10
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 49.04  |  12.91  | 7.0945 
 50  | 43.49  |  10.37  | 13.5076 
 75  | 38.66  |  9.42  | 19.9164 
100  | 35.63  |  8.48  | 26.4617 
125  | 32.47  |  7.79  | 32.8965 
150  | 29.51  |  8.06  | 39.3180 
175  | 27.39  |  6.84  | 45.7114 
200  | 25.34  |  6.35  | 52.1321 
225  | 23.26  |  5.78  | 58.5552 
250  | 22.80  |  5.43  | 65.2859 
275  | 20.60  |  5.31  | 71.7006 
300  | 19.31  |  4.70  | 78.8722 
325  | 18.81  |  4.78  | 85.2285 
350  | 17.92  |  4.29  | 91.6391 
375  | 17.92  |  4.23  | 98.1366 
383  | 16.91  |  4.00  | 100.1747 
101.063765 seconds (308.30 k allocations: 143.500 GB, 6.29% gc time)


In [95]:
# Testing Hadamard Sketch
method_name = "Hadamard";# ROS
options.sketchsize =2; # Int(floor(log2(prob.n)));
options.maxiter=200;
options.skip_error_calculation =25;
@time  output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

flopsperiter: 20008
Hadamard
-------------------
It   | Error% | Residual |  Time   
-------------------
  7  | 99.95  |  100.35  | 15.0460 
 15.151344 seconds (165.23 M allocations: 19.126 GB, 56.21% gc time)


In [88]:
# Testing countmin 
# options.sketchsize =floor((prob.n)^(1/2));
# options.skip_error_calculation =25;
# method_name = "countmin";# gradient descent
# @time output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [89]:
# Testing CG 
options.skip_error_calculation =25;
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 3.35  |  1.34  | 0.3300 
 50  | 1.57  |  1.42  | 1.1144 
 75  | 0.98  |  0.37  | 1.7181 
100  | 0.71  |  0.28  | 2.2727 
125  | 0.54  |  0.19  | 2.8231 
150  | 0.43  |  0.19  | 3.3589 
175  | 0.35  |  0.47  | 3.9264 
200  | 0.29  |  0.25  | 4.4865 
225  | 0.24  |  0.16  | 5.0502 
250  | 0.20  |  0.18  | 5.6142 
275  | 0.42  |  0.13  | 6.1760 
300  | 0.14  |  0.08  | 6.7436 
325  | 0.65  |  0.07  | 7.3063 
350  | 0.11  |  0.11  | 7.8692 
375  | 0.08  |  0.10  | 8.4237 
400  | 0.04  |  0.08  | 8.9737 
425  | 0.07  |  0.19  | 9.5284 
450  | 0.06  |  0.08  | 10.0782 
475  | 0.02  |  0.05  | 10.6280 
500  | 0.05  |  0.04  | 11.1814 
525  | 0.04  |  0.06  | 11.7136 
550  | 0.04  |  0.05  | 12.2737 
575  | 0.04  |  0.04  | 12.7543 
600  | 0.03  |  0.04  | 13.1623 
625  | 0.03  |  0.06  | 13.5745 
650  | 0.03  |  0.11  | 13.9933 
675  | 0.03  |  0.04  | 14.4046 
700  | 0.08  |  0.03  | 14.8152 
725  | 0.02  |

In [90]:
# Testing CD 
method_name = "CD";# gradient descent
options.sketchsize = floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-70
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 21.32  |  5.41  | 0.1719 
 50  | 14.60  |  3.56  | 0.3389 
 75  | 10.65  |  2.24  | 0.5064 
100  | 8.95  |  2.05  | 0.6716 
125  | 7.98  |  1.92  | 0.8388 
150  | 6.03  |  1.54  | 1.0082 
175  | 5.05  |  1.27  | 1.1740 
200  | 4.81  |  1.25  | 1.3405 
225  | 4.34  |  1.05  | 1.5052 
250  | 3.94  |  0.94  | 1.6751 
275  | 3.62  |  0.92  | 1.8416 
300  | 3.37  |  0.87  | 2.0132 
325  | 3.17  |  0.74  | 2.1784 
350  | 2.98  |  0.66  | 2.3456 
375  | 2.80  |  0.62  | 2.5205 
400  | 2.61  |  0.60  | 2.6964 
425  | 2.50  |  0.57  | 2.8659 
443  | 2.45  |  0.58  | 3.0049 
  3.704944 seconds (33.16 k allocations: 1.196 GB, 2.60% gc time)


In [93]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [94]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

gisette_scale


In [16]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];